# Stock Screener

## Tech Stack
- Python
- langchain
- langgraph

In [ ]:
# install dependencies
%pip install langgraph langchain openai \
    huggingface-hub langchain-community \
    


In [ ]:
# import dependencies
import json
import yfinance as yf
from typing import Annotated
from colorama import Fore
from langgraph.graph import START, END, StateGraph
from langgraph.grpah.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode
from langchain_ollama import ChatOllama
from langchain.tools import tool


In [ ]:
@tool
def simple_screener(screen_type:str, offset:int)-> str: 
    """Returns screened assets (stocks, funds, bonds) given popular criteria. 

    Args:
        screen_type: One of a default set of stock screener queries from yahoo finance. 
        aggressive_small_caps
        day_gainers
        day_losers
        growth_technology_stocks
        most_actives
        most_shorted_stocks
        small_cap_gainers
        undervalued_growth_stocks
        undervalued_large_caps
        conservative_foreign_funds
        high_yield_bond
        portfolio_anchors
        solid_large_growth_funds
        solid_midcap_growth_funds
        top_mutual_funds
      offset: the pagination start point

    Returns:
        The a JSON output of assets that meet the criteria
        """
    
    query = yf.PREDEFINED_SCREENER_QUERIES[screen_type]['query']
    result = yf.screen(query, offset=offset, size=5) 

    with open('output.json', 'w') as f: 
          json.dump(result, f) 
     
    fields = ["shortName","bid","ask","exchange", "fiftyTwoWeekHigh", "fiftyTwoWeekLow", "averageAnalystRating", "dividendYield", "symbol"] 
    output_data = []
    for stock_detail in result['quotes']: 
        details = {}
        for key, val in stock_detail.items(): 
            if key in fields: 
                details[key] = val 
        output_data.append(details) 
    
    return f"Stock Screener Results: {output_data}"

In [ ]:
# create LLM
llm = ChatOllama(model="qwen2.5:14b")
# create tools
tools = [simple_screener]
# bind llm w/ tools
master = llm.bind_tools(tools)
# create Tool Node
tool_node = ToolNode(tools)


In [ ]:
# create state
class State(dict):
    messages: Annotated[list, add_messages]

# build llm node
def chatbot(state: State):
    print(state["messages"])
    return {
        "messages": [  master.invoke(state["messages"]) ]
    }

# create router node
def router(state: State):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    else:
        return END


In [ ]:
# assemble graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge(START, "chatbot")
# update graph for Tools
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", router)
# add memory and compile graph
memory = InMemorySaver()
graph = graph_builder.compile(check_pointer=memory)


# build call loop and run it
while True:
    prompt = input(" ")
    result = graph.invoke(
        { "messages": [{
                "role": "user", 
                "content": prompt 
            }],
        }, 
        config= {
            "configurable":{ 
                "thread_id": 1234
            },
        },
    )
    
    print(Fore.LIGHTYELLOW_EX + result["messages"][-1].content + Fore.RESET)